In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# Import all necessary libraries.
import numpy as np
import sys
import time
from turbojpeg import TurboJPEG
jpeg = TurboJPEG()

import warnings
warnings.filterwarnings('ignore')

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [3]:
from nomeroff_net.pipes.number_plate_localizators.yolo_v5_detector import Detector

detector = Detector()
detector.load()

YOLOv5 🚀 v6.0-244-g9cf80b7 torch 1.10.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11019MiB)

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs


In [4]:
from nomeroff_net.pipes.number_plate_keypoints_detectors.bbox_np_points import NpPointsCraft
from nomeroff_net.tools import convert_cv_zones_rgb_to_bgr, get_cv_zone_rgb, reshape_points

npPointsCraft = NpPointsCraft()
npPointsCraft.load()

Loading weights from checkpoint (/mnt/store/nomeroff-net/nomeroff_net/tools/../../data/./models/np_points_craft/craft_mlt/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/mnt/store/nomeroff-net/nomeroff_net/tools/../../data/./models/np_points_craft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth)


In [5]:
from nomeroff_net.pipes.number_plate_classificators.options_detector import OptionsDetector

optionsDetector = OptionsDetector()
optionsDetector.load()

NPOptionsNet(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

In [6]:
from nomeroff_net.pipes.number_plate_text_readers.text_detector import TextDetector

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "latest"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "latest"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "latest"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-ordlo-lpr", "eu-ua-ordlo-dpr"],
        "model_path": "latest" 
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "latest"
    },
    "kg": {
        "for_regions": ["kg"],
        "model_path": "latest"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "latest"
    },
    "su": {
        "for_regions": ["su"],
        "model_path": "latest"
    },
    "am": {
        "for_regions": ["am"],
        "model_path": "latest"
    },
})

In [7]:
async def test(dirName, fname):
    start_time = time.time()
    img_path = os.path.join(dirName, fname)
    with open(img_path, 'rb') as in_file:
        img = jpeg.decode(in_file.read())
    #img = cv2.imread(img_path)
    image_load_time = time.time() - start_time

    start_time = time.time()
    targetBoxes = detector.detect_bbox(img)
    detect_bbox_time = time.time() - start_time
    
    start_time = time.time()
    all_points = npPointsCraft.detect(img, targetBoxes)
    all_points = [ps for ps in all_points if len(ps)]
    craft_time = time.time() - start_time
    
    start_time = time.time()
    
    zones = convert_cv_zones_rgb_to_bgr([get_cv_zone_rgb(img, reshape_points(rect,1)) for rect in all_points])
    
    perspective_align_time = time.time() - start_time
    
    start_time = time.time()
    regionIds, countLines = optionsDetector.predict(zones)
    regionNames = optionsDetector.get_region_labels(regionIds)
    classification_time = time.time() - start_time
    
    start_time = time.time()
    #print(fname, regionNames, countLines)
    #textArr = textDetector.predict(zones, regionNames, countLines)
    model_inputs = textDetector.preprocess(zones, regionNames, countLines)
    model_outputs = textDetector.forward(model_inputs)
    #textArr = self.ocr_detector.postprocess(model_outputs)
    ocr_time = time.time() - start_time
    return (image_load_time, detect_bbox_time, craft_time, perspective_align_time, classification_time, ocr_time)

In [12]:
N = 10

i = 0
j = 0

image_load_time_all        = 0
detect_bbox_time_all       = 0
craft_time_all             = 0
perspective_align_time_all = 0
classification_time_all    = 0
ocr_time_all               = 0

start_time = time.time()
rootDir = 'images/'
for i in np.arange(N):
    print("pass {}".format(i))
    for dirName, subdirList, fileList in os.walk(rootDir):
        for fname in fileList:
            image_load_time, \
                detect_bbox_time, \
                craft_time, \
                perspective_align_time, \
                classification_time, \
                ocr_time = await test(dirName, fname)
            image_load_time_all        += image_load_time
            detect_bbox_time_all       += detect_bbox_time
            craft_time_all             += craft_time
            perspective_align_time_all += perspective_align_time
            classification_time_all    += classification_time
            ocr_time_all               += ocr_time
            j += 1
            #print(i, j)
    i += 1
end_time = time.time() - start_time

pass 0
pass 1
pass 2
pass 3
pass 4
pass 5
pass 6
pass 7
pass 8
pass 9


In [13]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")
print()
print(f"image_load_time_all {image_load_time_all}; {image_load_time_all/j} per one photo")
print(f"detect_bbox_time_all {detect_bbox_time_all}; {detect_bbox_time_all/j} per one photo")
print(f"craft_time_all {craft_time_all}; {craft_time_all/j} per one photo")
print(f"perspective_align_time_all {perspective_align_time_all}; {perspective_align_time_all/j} per one photo")
print(f"classification_time_all {classification_time_all}; {classification_time_all/j} per one photo")
print(f"ocr_time_all {ocr_time_all}; {ocr_time_all/j} per one photo")

Processed 250 photos
Time 14.905659914016724
One photo process 0.05962263965606689 seconds

image_load_time_all 5.617801666259766; 0.022471206665039063 per one photo
detect_bbox_time_all 2.6758947372436523; 0.010703578948974609 per one photo
craft_time_all 4.165661096572876; 0.016662644386291503 per one photo
perspective_align_time_all 0.7589242458343506; 0.0030356969833374024 per one photo
classification_time_all 0.7444291114807129; 0.0029777164459228516 per one photo
ocr_time_all 0.9303250312805176; 0.00372130012512207 per one photo


In [ ]:
# old version

# Processed 310 photos
# Time 15.948540687561035
# One photo process 0.05144690544374528 seconds

# image_load_time_all 6.033987283706665; 0.01946447510873118 per one photo
# detect_bbox_time_all 2.778740644454956; 0.008963679498241794 per one photo
# craft_time_all 4.546112775802612; 0.01466487992194391 per one photo
# perspective_align_time_all 0.9103446006774902; 0.00293659548605642 per one photo
# classification_time_all 0.7576234340667725; 0.0024439465615057176 per one photo
# ocr_time_all 0.908959150314331; 0.002932126291336552 per one photo